In [1]:
#imports and packages
!pip install imutils
!pip install matplotlib

In [62]:
from imutils import paths
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

#parameters
run_on_all_images = False            # Run the script over all images from a query set
image_names = ["00001.jpg","00002.jpg"]          # Names of images to run the script on in case the script is not run on all images
query_set = 1                        # Which query set of images to run the script on (1 = Q1, 2 = Q2)
grayscale_method = 'a'               # Method used to convert image to grayscale (a = average, w = weighted)
similarity_method = 'e'              # The equation used for calculating similarity measures (e = euclidean distance, l = L1 distance, x = X square distance, i = histogram intersection)
input_folder_q1 = '../Data/qsd1_w1'  # Path to the folder containing the first query set of images
input_folder_q2 = '../Data/qsd2_w1'  # Path to the folder containing the second query set of images
input_folder_bd = '../Data/bbdd'     # Path tp the folder containing the image database (comparison images)
output = 10                          # Number of results to be returned

In [32]:
# utils

def get_grayscale(image):
    R, G, B = image[:,:,0], image[:,:,1], image[:,:,2]
    if grayscale_method == 'a':
        img_gray = (R + G + B) / 3 
    else:
        img_gray = 0.2126 * R + 0.7152 * G + 0.0722 * B
    return np.float32(img_gray)

def get_histogram(image):
    return cv2.calcHist([image],[0],None,[256],[0,256])

def get_similarity(query_histogram, bd_histogram):
    distance = 0
    if similarity_method == 'e':
        distance = euclidian_distance_comparison(query_histogram, bd_histogram)
    elif similarity_method == 'l':
        distance = l1_distance_comparison(query_histogram, bd_histogram)
    elif similarity_method == 'x':
        distance = x_square_distance_comparison(query_histogram, bd_histogram)
    elif similarity_method == 'i':
        distance = intersection_comparison(query_histogram, bd_histogram)
    return distance

def euclidian_distance_comparison(query_histogram, bd_histogram):
    dist = 0
    for i in range(256):
        dist += (query_histogram[i] - bd_histogram[i])**2
    return math.sqrt(dist)

def l1_distance_comparison(query_histogram, bd_histogram):
    dist = 0
    for i in range(256):
        dist += abs(query_histogram[i] - bd_histogram[i])
    return dist

def x_square_distance_comparison(query_histogram, bd_histogram):
    dist = 0
    for i in range(256):
        dist += (query_histogram[i] - bd_histogram[i])**2/(query_histogram[i] + bd_histogram[i])
    return dist

def intersection_comparison(query_histogram, bd_histogram):
    return cv2.compareHist(query_histogram, bd_histogram, cv2.HISTCMP_INTERSECT)


In [63]:
histograms = {}
similarities = {}
sorted_similarities = {}
result = []
input_folder = input_folder_q1
if query_set == 2:
    input_folder = input_folder_q2

# Get image paths (either all or for the given names)
query_image_paths = []
if run_on_all_images:
    query_image_paths = list(paths.list_images(input_folder))
else:
    for image_name in image_names:
        query_image_paths.append(input_folder + '/' + image_name)
        
# Get data base image paths
bd_image_paths = list(paths.list_images(input_folder_bd))
for bd_image_path in bd_image_paths:
    image = cv2.imread(bd_image_path)
    img_gray = get_grayscale(image)
    type(histograms)
    histograms[bd_image_path] = get_histogram(img_gray)


# Traverse query images and calculate similarities
for query_image_path in query_image_paths:
    image = cv2.imread(query_image_path)
    img_gray = get_grayscale(image)
    histograms[query_image_path] = get_histogram(img_gray)
    similarities[query_image_path] = []
    for bd_image_path in bd_image_paths:
        similarity = get_similarity(histograms[query_image_path], histograms[bd_image_path])
        similarities[query_image_path].append((bd_image_path, similarity))
    # Sort similarities array by the second value in the tuples (similarity calculation)
    sorted_similarities[query_image_path] = sorted(list(similarities[query_image_path]), key=lambda tup: tup[1])
    
    # Results
    result.append([tup[0] for tup in list(sorted_similarities[query_image_path])[:output]])
#TODO: extract int from the file name in the path
print(result)

[['../Data/bbdd\\bbdd_00176.jpg', '../Data/bbdd\\bbdd_00124.jpg', '../Data/bbdd\\bbdd_00172.jpg', '../Data/bbdd\\bbdd_00047.jpg', '../Data/bbdd\\bbdd_00074.jpg', '../Data/bbdd\\bbdd_00148.jpg', '../Data/bbdd\\bbdd_00005.jpg', '../Data/bbdd\\bbdd_00122.jpg', '../Data/bbdd\\bbdd_00065.jpg', '../Data/bbdd\\bbdd_00068.jpg'], ['../Data/bbdd\\bbdd_00246.jpg', '../Data/bbdd\\bbdd_00133.jpg', '../Data/bbdd\\bbdd_00157.jpg', '../Data/bbdd\\bbdd_00002.jpg', '../Data/bbdd\\bbdd_00229.jpg', '../Data/bbdd\\bbdd_00212.jpg', '../Data/bbdd\\bbdd_00049.jpg', '../Data/bbdd\\bbdd_00208.jpg', '../Data/bbdd\\bbdd_00242.jpg', '../Data/bbdd\\bbdd_00115.jpg']]


In [ ]:
Tot = len(imagePaths)
Cols = 5
Rows = Tot // Cols 

if Tot % Cols != 0:
    Rows += 1

Position = range(1,Tot + 1)